In [ ]:
import pandas as pd
import tkinter as tk
import pyodbc
import calendar
import warnings
import locale
import os
from tkinter import messagebox
from datetime import datetime

warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

header = [
    'PERIODO', 'AGENCIA', 'CC', 'CONTRATO', 'CARTERA', 'PRIORIDAD', 'CAPITAL', 'RECUPERO', 'SEGMENTO', 'PRODUCTO', 
    'INTENSIDAD_TOTAL', 'INTENSIDAD_DIRECTA', 'COBERTURA', 'PDP', 'PDP_CUMPLIDA'
]


def get_connection(server_name: str, database_name: str, username: str = None, password: str = None) -> pyodbc.Connection:
    try:
        if username and password:
            conn = pyodbc.connect(
                'DRIVER={ODBC Driver 17 for SQL Server};'
                f'SERVER={server_name};'
                f'DATABASE={database_name};'
                f'UID={username};'
                f'PWD={password};'
            )
            i = 0
        else:
            conn = pyodbc.connect(
                'DRIVER={ODBC Driver 17 for SQL Server};'
                f'SERVER={server_name};'
                f'DATABASE={database_name};'
                'Trusted_Connection=yes;'
            )
            i = 1
    except Exception as e:
        print('Error de conexión:', str(e))
        return
    finally:
        if conn:
            if i == 1:
                print('Conexión exitosa: Autenticación de Windows')
            else:
                print('Conexión exitosa: UserCobranza')
            return conn


def obtener_periodo(mes_anterior=False):
    hoy = datetime.now()
    
    if mes_anterior:
        if hoy.month == 1:
            mes = 12
            año = hoy.year - 1
        else:
            mes = hoy.month - 1
            año = hoy.year
    else:
        mes = hoy.month
        año = hoy.year
    
    return f'{año}{str(mes).zfill(2)}'


def obtener_mes(anterior=False):
    hoy = datetime.now()
    
    if anterior:
        if hoy.month == 1:
            mes = 12
        else:
            mes = hoy.month - 1
    else:
        mes = hoy.month
    
    return calendar.month_name[mes].upper()


def obtener_periodos_año(año: int, hasta_mes_actual: bool = True) -> list[str]:
    hoy = datetime.now()
    periodos = []
    
    if año == hoy.year and hasta_mes_actual:
        for mes in range(1, hoy.month + 1):
            periodo = f'{año}{str(mes).zfill(2)}'
            periodos.append(periodo)
    else:
        for mes in range(1, 13):
            periodo = f'{año}{str(mes).zfill(2)}'
            periodos.append(periodo)
    
    return periodos


def leer_archivos_csv(periodos: str | list[str], base_path: str) -> dict[str, pd.DataFrame]:
    dataframes = {}
    
    if isinstance(periodos, str):
        periodos = [periodos]
    
    for periodo in periodos:
        file_path = f'{base_path}/{periodo}/Efectividades_{periodo}.csv'
        if os.path.exists(file_path):
            df = pd.read_csv(
                file_path, 
                sep=';', 
                encoding='utf-8', 
                #header=None
            )
            #df.columns = header
            print(f'Periodo: {periodo} - {df.shape}')
            dataframes[periodo] = df
        else:
            print(f'Archivo no encontrado: {file_path}')
    
    return dataframes


def dashboard_efectividades(df: pd.DataFrame) -> pd.DataFrame:  
    df = df[['PERIODO', 'AGENCIA', 'CARTERA', 'PRIORIDAD', 'CAPITAL', 'RECUPERO']]
    df = df[
        (df['AGENCIA'].isin(['ASESCOM RJ', 'CLASA MORA', 'MORNESE MORA']))
        & (df['CARTERA'].isin(['BC', 'EXJ', 'KSTBC']))
        & (df['PRIORIDAD'].isin(['C1', 'C2', 'C3']))
    ]
    
    df['CAPITAL'] = df['CAPITAL'].round(2)
    df['RECUPERO'] = df['RECUPERO'].round(2)
    df['PRIORIDAD'] = df['PRIORIDAD'].fillna('NULL')
    
    df = df.groupby(['PERIODO', 'AGENCIA', 'CARTERA', 'PRIORIDAD']).agg({'RECUPERO': 'sum', 'CAPITAL': 'sum'}).reset_index()
    
    df.sort_values(by=['PERIODO', 'AGENCIA', 'CARTERA', 'PRIORIDAD'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df


def dashboard_saldos(df: pd.DataFrame) -> pd.DataFrame:
    df = df[['PERIODO', 'AGENCIA', 'CARTERA', 'PRIORIDAD', 'CAPITAL', 'SEGMENTO', 'PRODUCTO']]
    df = df[
        (df['AGENCIA'].isin(['ASESCOM RJ', 'CLASA MORA', 'MORNESE MORA', 'SIN AGENCIA']))
        & (df['CARTERA'].isin(['BC', 'EXJ', 'KSTBC']))
        & (df['PRIORIDAD'].isin(['C1', 'C2', 'C3']))
    ]
    
    df['SEGMENTO'] = df['SEGMENTO'].apply(lambda x: 'PYME / EMP MIN' if x != 'PARTICULARES' else x)
    df['PRODUCTO'] = df['PRODUCTO'].astype(str).str.replace(r'\d+', '', regex=True).str.replace('.', '').str.upper()
    df['PRODUCTO'] = df['PRODUCTO'].astype(str).str.strip().replace(' ', '')
    df['CAPITAL'] = df['CAPITAL'].round(2)
    df['PRIORIDAD'] = df['PRIORIDAD'].fillna('NULL')
    
    df = df.groupby(['PERIODO', 'AGENCIA', 'CARTERA', 'PRIORIDAD', 'SEGMENTO', 'PRODUCTO']).agg({'CAPITAL': 'sum'}).reset_index()
    
    df.sort_values(by=['PERIODO', 'AGENCIA', 'CARTERA', 'PRIORIDAD', 'SEGMENTO', 'PRODUCTO'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df


def dashboard_indicadores(df: pd.DataFrame) -> pd.DataFrame:
    df = df[['PERIODO', 'AGENCIA', 'CC', 'CONTRATO', 'CARTERA', 'PRIORIDAD', 'INTENSIDAD_TOTAL', 'INTENSIDAD_DIRECTA', 'COBERTURA', 'PDP', 'PDP_CUMPLIDA']]
    df = df[
        (df['AGENCIA'].isin(['ASESCOM RJ', 'CLASA MORA', 'MORNESE MORA']))
        & (df['CARTERA'].isin(['BC', 'EXJ', 'KSTBC']))
        & (df['PRIORIDAD'].isin(['C1', 'C2', 'C3']))
    ]
    
    df['INTENSIDAD_TOTAL'] = df['INTENSIDAD_TOTAL'].fillna(0)
    df['INTENSIDAD_DIRECTA'] = df['INTENSIDAD_DIRECTA'].fillna(0)
    df['COBERTURA'] = df['COBERTURA'].fillna(0)
    df['PDP'] = df['PDP'].fillna(0)
    df['PDP_CUMPLIDA'] = df['PDP_CUMPLIDA'].fillna(0)
    
    cols_order = ['PERIODO', 'AGENCIA', 'CARTERA', 'PRIORIDAD', 'INTENSIDAD_TOTAL', 'INTENSIDAD_DIRECTA', 'COBERTURA']
    
    # Conteo diferenciado de TOTAL - PRIORIDAD
    df_conteo_dif_total_prioridad = df.groupby(['AGENCIA', 'PRIORIDAD']).agg(
        CONTEO_DIFERENCIADO=('CC', 'nunique'), 
        CONTEO_DIFERENCIADO_CD=('CC', lambda x: x[df['INTENSIDAD_DIRECTA'] != 0].nunique()), 
        CONTEO_DIFERENCIADO_PDP=('CC', lambda x: x[df['PDP'] == 1].nunique()), 
        CONTEO_DIFERENCIADO_PDP_CUMPLIDA=('CC', lambda x: x[df['PDP_CUMPLIDA'] == 1].nunique())
    ).reset_index()
    
    df_agrupado_total_prioridad = df.groupby(['PERIODO', 'AGENCIA', 'PRIORIDAD']).agg({
        'INTENSIDAD_TOTAL': 'sum', 
        'INTENSIDAD_DIRECTA': 'sum', 
        'COBERTURA': 'mean'
    }).reset_index()
    
    df_agrupado_total_prioridad['CARTERA'] = 'TOTAL'
    df_agrupado_total_prioridad = df_agrupado_total_prioridad[cols_order]
    
    df_final_total_prioridad = pd.merge(df_agrupado_total_prioridad, df_conteo_dif_total_prioridad, on=['AGENCIA', 'PRIORIDAD'], how='left')
    df_final_total_prioridad['INTENSIDAD_TOTAL'] = df_final_total_prioridad['INTENSIDAD_TOTAL']/df_final_total_prioridad['CONTEO_DIFERENCIADO']
    df_final_total_prioridad['INTENSIDAD_DIRECTA'] = df_final_total_prioridad['INTENSIDAD_DIRECTA']/df_final_total_prioridad['CONTEO_DIFERENCIADO_CD']
    df_final_total_prioridad['CONTACTO_DIRECTO'] = df_final_total_prioridad['CONTEO_DIFERENCIADO_CD']/df_final_total_prioridad['CONTEO_DIFERENCIADO']
    df_final_total_prioridad['TASA_CIERRE'] = df_final_total_prioridad['CONTEO_DIFERENCIADO_PDP']/df_final_total_prioridad['CONTEO_DIFERENCIADO_CD']
    df_final_total_prioridad['CUMPLIMIENTO_PDP'] = df_final_total_prioridad['CONTEO_DIFERENCIADO_PDP_CUMPLIDA']/df_final_total_prioridad['CONTEO_DIFERENCIADO_PDP']
    df_final_total_prioridad.drop(columns=['CONTEO_DIFERENCIADO', 'CONTEO_DIFERENCIADO_CD', 'CONTEO_DIFERENCIADO_PDP', 'CONTEO_DIFERENCIADO_PDP_CUMPLIDA'], inplace=True)
    
    # Conteo diferenciado de CARTERA - TOTAL
    df_conteo_dif_cartera_total = df.groupby(['AGENCIA', 'CARTERA']).agg(
        CONTEO_DIFERENCIADO=('CC', 'nunique'), 
        CONTEO_DIFERENCIADO_CD=('CC', lambda x: x[df['INTENSIDAD_DIRECTA'] != 0].nunique()), 
        CONTEO_DIFERENCIADO_PDP=('CC', lambda x: x[df['PDP'] == 1].nunique()), 
        CONTEO_DIFERENCIADO_PDP_CUMPLIDA=('CC', lambda x: x[df['PDP_CUMPLIDA'] == 1].nunique())
    ).reset_index()
    
    df_agrupado_cartera_total = df.groupby(['PERIODO', 'AGENCIA', 'CARTERA']).agg({
        'INTENSIDAD_TOTAL': 'sum', 
        'INTENSIDAD_DIRECTA': 'sum', 
        'COBERTURA': 'mean'
    }).reset_index()
    
    df_agrupado_cartera_total['PRIORIDAD'] = 'TOTAL'
    df_agrupado_cartera_total = df_agrupado_cartera_total[cols_order]
    
    df_final_cartera_total = pd.merge(df_agrupado_cartera_total, df_conteo_dif_cartera_total, on=['AGENCIA', 'CARTERA'], how='left')
    df_final_cartera_total['INTENSIDAD_TOTAL'] = df_final_cartera_total['INTENSIDAD_TOTAL']/df_final_cartera_total['CONTEO_DIFERENCIADO']
    df_final_cartera_total['INTENSIDAD_DIRECTA'] = df_final_cartera_total['INTENSIDAD_DIRECTA']/df_final_cartera_total['CONTEO_DIFERENCIADO_CD']
    df_final_cartera_total['CONTACTO_DIRECTO'] = df_final_cartera_total['CONTEO_DIFERENCIADO_CD']/df_final_cartera_total['CONTEO_DIFERENCIADO']
    df_final_cartera_total['TASA_CIERRE'] = df_final_cartera_total['CONTEO_DIFERENCIADO_PDP']/df_final_cartera_total['CONTEO_DIFERENCIADO_CD']
    df_final_cartera_total['CUMPLIMIENTO_PDP'] = df_final_cartera_total['CONTEO_DIFERENCIADO_PDP_CUMPLIDA']/df_final_cartera_total['CONTEO_DIFERENCIADO_PDP']
    df_final_cartera_total.drop(columns=['CONTEO_DIFERENCIADO', 'CONTEO_DIFERENCIADO_CD', 'CONTEO_DIFERENCIADO_PDP', 'CONTEO_DIFERENCIADO_PDP_CUMPLIDA'], inplace=True)
    
    # Conteo diferenciado de TOTAL - TOTAL
    df_conteo_dif_total_total = df.groupby(['AGENCIA']).agg(
        CONTEO_DIFERENCIADO=('CC', 'nunique'), 
        CONTEO_DIFERENCIADO_CD=('CC', lambda x: x[df['INTENSIDAD_DIRECTA'] != 0].nunique()), 
        CONTEO_DIFERENCIADO_PDP=('CC', lambda x: x[df['PDP'] == 1].nunique()), 
        CONTEO_DIFERENCIADO_PDP_CUMPLIDA=('CC', lambda x: x[df['PDP_CUMPLIDA'] == 1].nunique())
    ).reset_index()
    
    df_agrupado_total_total = df.groupby(['PERIODO', 'AGENCIA']).agg({
        'INTENSIDAD_TOTAL': 'sum', 
        'INTENSIDAD_DIRECTA': 'sum', 
        'COBERTURA': 'mean'
    }).reset_index()
    
    df_agrupado_total_total['CARTERA'] = 'TOTAL'
    df_agrupado_total_total['PRIORIDAD'] = 'TOTAL'
    df_agrupado_total_total = df_agrupado_total_total[cols_order]
    
    df_final_total_total = pd.merge(df_agrupado_total_total, df_conteo_dif_total_total, on=['AGENCIA'], how='left')
    df_final_total_total['INTENSIDAD_TOTAL'] = df_final_total_total['INTENSIDAD_TOTAL']/df_final_total_total['CONTEO_DIFERENCIADO']
    df_final_total_total['INTENSIDAD_DIRECTA'] = df_final_total_total['INTENSIDAD_DIRECTA']/df_final_total_total['CONTEO_DIFERENCIADO_CD']
    df_final_total_total['CONTACTO_DIRECTO'] = df_final_total_total['CONTEO_DIFERENCIADO_CD']/df_final_total_total['CONTEO_DIFERENCIADO']
    df_final_total_total['TASA_CIERRE'] = df_final_total_total['CONTEO_DIFERENCIADO_PDP']/df_final_total_total['CONTEO_DIFERENCIADO_CD']
    df_final_total_total['CUMPLIMIENTO_PDP'] = df_final_total_total['CONTEO_DIFERENCIADO_PDP_CUMPLIDA']/df_final_total_total['CONTEO_DIFERENCIADO_PDP']
    df_final_total_total.drop(columns=['CONTEO_DIFERENCIADO', 'CONTEO_DIFERENCIADO_CD', 'CONTEO_DIFERENCIADO_PDP', 'CONTEO_DIFERENCIADO_PDP_CUMPLIDA'], inplace=True)
    
    # Conteo diferenciado de CARTERA - PRIORIDAD
    df_conteo_dif_cartera_prioridad = df.groupby(['AGENCIA', 'CARTERA', 'PRIORIDAD']).agg(
        CONTEO_DIFERENCIADO=('CC', 'nunique'), 
        CONTEO_DIFERENCIADO_CD=('CC', lambda x: x[df['INTENSIDAD_DIRECTA'] != 0].nunique()), 
        CONTEO_DIFERENCIADO_PDP=('CC', lambda x: x[df['PDP'] == 1].nunique()), 
        CONTEO_DIFERENCIADO_PDP_CUMPLIDA=('CC', lambda x: x[df['PDP_CUMPLIDA'] == 1].nunique())
    ).reset_index()
    
    df = df.groupby(['PERIODO', 'AGENCIA', 'CARTERA', 'PRIORIDAD']).agg({
        'INTENSIDAD_TOTAL': 'sum', 
        'INTENSIDAD_DIRECTA': 'sum', 
        'COBERTURA': 'mean'
    }).reset_index()
    
    df = pd.merge(df, df_conteo_dif_cartera_prioridad, on=['AGENCIA', 'CARTERA', 'PRIORIDAD'], how='left')
    df['INTENSIDAD_TOTAL'] = df['INTENSIDAD_TOTAL']/df['CONTEO_DIFERENCIADO']
    df['INTENSIDAD_DIRECTA'] = df['INTENSIDAD_DIRECTA']/df['CONTEO_DIFERENCIADO_CD']
    df['CONTACTO_DIRECTO'] = df['CONTEO_DIFERENCIADO_CD']/df['CONTEO_DIFERENCIADO']
    df['TASA_CIERRE'] = df['CONTEO_DIFERENCIADO_PDP']/df['CONTEO_DIFERENCIADO_CD']
    df['CUMPLIMIENTO_PDP'] = df['CONTEO_DIFERENCIADO_PDP_CUMPLIDA']/df['CONTEO_DIFERENCIADO_PDP']
    df.drop(columns=['CONTEO_DIFERENCIADO', 'CONTEO_DIFERENCIADO_CD', 'CONTEO_DIFERENCIADO_PDP', 'CONTEO_DIFERENCIADO_PDP_CUMPLIDA'], inplace=True)
    
    # Unión de CARTERA - PRIORIDAD, CARTERA - TOTAL, TOTAL - PRIORIDAD y TOTAL - TOTAL
    df_final = pd.concat([df, df_final_total_prioridad, df_final_cartera_total, df_final_total_total], ignore_index=True)
    
    df_final['INTENSIDAD_TOTAL'] = df_final['INTENSIDAD_TOTAL'].round(2)
    df_final['INTENSIDAD_DIRECTA'] = df_final['INTENSIDAD_DIRECTA'].round(2)
    df_final['COBERTURA'] = df_final['COBERTURA'].round(4)
    df_final['CONTACTO_DIRECTO'] = df_final['CONTACTO_DIRECTO'].round(4)
    df_final['TASA_CIERRE'] = df_final['TASA_CIERRE'].round(4)
    df_final['CUMPLIMIENTO_PDP'] = df_final['CUMPLIMIENTO_PDP'].round(4)
    
    df_final.sort_values(by=['PERIODO', 'AGENCIA', 'CARTERA', 'PRIORIDAD'], inplace=True)
    df_final.reset_index(drop=True, inplace=True)
    
    return df_final[['PERIODO', 'AGENCIA', 'CARTERA', 'PRIORIDAD', 'INTENSIDAD_TOTAL', 'INTENSIDAD_DIRECTA', 'COBERTURA', 'CONTACTO_DIRECTO', 'TASA_CIERRE', 'CUMPLIMIENTO_PDP']]


def dashboard_bench(df: pd.DataFrame, df_metas: pd.DataFrame) -> pd.DataFrame:
    # Bench Cartera - Agencia
    df_bench = df[
        (df['AGENCIA'].isin(['ASESCOM RJ', 'CLASA MORA', 'MORNESE MORA']))
        & (df['CARTERA'].isin(['BC', 'EXJ', 'KSTBC']))
        & (df['PRIORIDAD'].isin(['C1', 'C2', 'C3']))
    ]
    
    df_bench_agg = df_bench.groupby(['PERIODO', 'CARTERA', 'AGENCIA']).agg({
        'CAPITAL': 'sum',
        'CONTRATO': lambda x: x[df_bench['RECUPERO'] > 0].count(), 
        'RECUPERO': 'sum', 
    }).reset_index()
    
    df_bench_agg.rename(columns={'CONTRATO': 'N_PAGO'}, inplace=True)
    
    df_bench_agg['CAPITAL'] = df_bench_agg['CAPITAL'].round(2)
    df_bench_agg['RECUPERO'] = df_bench_agg['RECUPERO'].round(2)
    df_bench_agg['TICKET_PAGO'] = (df_bench_agg['RECUPERO'] / df_bench_agg['N_PAGO']).round(0).astype(int)
    
    df_bench_agg = df_bench_agg.merge(df_metas, on=['PERIODO', 'CARTERA', 'AGENCIA'], how='left')
    
    df_bench_agg['EFECTIVIDAD_META(%)'] = (df_bench_agg['META'] / df_bench_agg['CAPITAL']).astype(float)
    df_bench_agg['EFECTIVIDAD_REAL(%)'] = (df_bench_agg['RECUPERO'] / df_bench_agg['CAPITAL']).astype(float)
    df_bench_agg['CUMPLIMIENTO(%)'] = (df_bench_agg['RECUPERO'] / df_bench_agg['META']).astype(float)
    
    df_bench_agg = df_bench_agg[['PERIODO', 'CARTERA', 'AGENCIA', 'CAPITAL', 'META', 'N_PAGO', 'RECUPERO', 'TICKET_PAGO', 'EFECTIVIDAD_META(%)', 'EFECTIVIDAD_REAL(%)', 'CUMPLIMIENTO(%)']]
    df_bench_agg.sort_values(by=['PERIODO', 'CARTERA', 'AGENCIA'], inplace=True)
    
    # Cartera - Total
    df_bench_agg_cartera = df_bench_agg.groupby(['PERIODO', 'CARTERA']).agg({
        'CAPITAL': 'sum', 
        'META': 'sum', 
        'N_PAGO': 'sum', 
        'RECUPERO': 'sum'
    }).reset_index()
    
    df_bench_agg_cartera['TICKET_PAGO'] = (df_bench_agg_cartera['RECUPERO'] / df_bench_agg_cartera['N_PAGO']).round(0).astype(int)
    df_bench_agg_cartera['EFECTIVIDAD_META(%)'] = (df_bench_agg_cartera['META'] / df_bench_agg_cartera['CAPITAL']).astype(float)
    df_bench_agg_cartera['EFECTIVIDAD_REAL(%)'] = (df_bench_agg_cartera['RECUPERO'] / df_bench_agg_cartera['CAPITAL']).astype(float)
    df_bench_agg_cartera['CUMPLIMIENTO(%)'] = (df_bench_agg_cartera['RECUPERO'] / df_bench_agg_cartera['META']).astype(float)
    df_bench_agg_cartera['AGENCIA'] = 'TOTAL'
    
    df_bench_agg_cartera = df_bench_agg_cartera[['PERIODO', 'CARTERA', 'AGENCIA', 'CAPITAL', 'META', 'N_PAGO', 'RECUPERO', 'TICKET_PAGO', 'EFECTIVIDAD_META(%)', 'EFECTIVIDAD_REAL(%)', 'CUMPLIMIENTO(%)']]
    
    # Total - Total
    df_bench_agg_total = df_bench_agg.groupby(['PERIODO']).agg({
        'CAPITAL': 'sum', 
        'META': 'sum', 
        'N_PAGO': 'sum', 
        'RECUPERO': 'sum'
    }).reset_index()
    
    df_bench_agg_total['TICKET_PAGO'] = (df_bench_agg_total['RECUPERO'] / df_bench_agg_total['N_PAGO']).round(0).astype(int)
    df_bench_agg_total['EFECTIVIDAD_META(%)'] = (df_bench_agg_total['META'] / df_bench_agg_total['CAPITAL']).astype(float)
    df_bench_agg_total['EFECTIVIDAD_REAL(%)'] = (df_bench_agg_total['RECUPERO'] / df_bench_agg_total['CAPITAL']).astype(float)
    df_bench_agg_total['CUMPLIMIENTO(%)'] = (df_bench_agg_total['RECUPERO'] / df_bench_agg_total['META']).astype(float)
    df_bench_agg_total['CARTERA'] = 'TOTAL'
    df_bench_agg_total['AGENCIA'] = 'TOTAL'
    
    df_bench_agg_total = df_bench_agg_total[['PERIODO', 'CARTERA', 'AGENCIA', 'CAPITAL', 'META', 'N_PAGO', 'RECUPERO', 'TICKET_PAGO', 'EFECTIVIDAD_META(%)', 'EFECTIVIDAD_REAL(%)', 'CUMPLIMIENTO(%)']]
    
    df_bench_final_1 = pd.concat([df_bench_agg, df_bench_agg_cartera, df_bench_agg_total], ignore_index=True)
    df_bench_final_1.sort_values(by=['PERIODO', 'CARTERA', 'AGENCIA'], inplace=True)
    df_bench_final_1.reset_index(drop=True, inplace=True)
    
    # Bench Agencia - Cartera - Segmento
    df_bench_seg = df[(df['AGENCIA'].isin(['ASESCOM RJ', 'CLASA MORA', 'MORNESE MORA'])) & 
                                (df['CARTERA'].isin(['BC', 'EXJ', 'KSTBC'])) & 
                                (df['PRIORIDAD'].isin(['C1', 'C2', 'C3']))]
    
    df_bench_seg_agg = df_bench_seg.groupby(['PERIODO', 'AGENCIA', 'CARTERA', 'PRIORIDAD']).agg({
        'CAPITAL': 'sum',
        'CONTRATO': 'count', 
        'RECUPERO': 'sum', 
    }).reset_index()
    
    df_bench_seg_agg.rename(columns={'CONTRATO': 'CONTRATOS'}, inplace=True)
    
    df_bench_seg_agg['CAPITAL'] = (df_bench_seg_agg['CAPITAL']).round(2)
    df_bench_seg_agg['RECUPERO'] = (df_bench_seg_agg['RECUPERO']).round(2)
    
    df_bench_seg_agg['CONTRATOS(%)'] = (df_bench_seg_agg['CONTRATOS'] / df_bench_seg_agg.groupby(['AGENCIA', 'CARTERA'])['CONTRATOS'].transform('sum')).astype(float)
    df_bench_seg_agg['CAPITAL(%)'] = (df_bench_seg_agg['CAPITAL'] / df_bench_seg_agg.groupby(['AGENCIA', 'CARTERA'])['CAPITAL'].transform('sum')).astype(float)
    df_bench_seg_agg['RECUPERO(%)'] = (df_bench_seg_agg['RECUPERO'] / df_bench_seg_agg.groupby(['AGENCIA', 'CARTERA'])['RECUPERO'].transform('sum')).astype(float)
    
    df_bench_seg_agg['EFECTIVIDAD(%)'] = (df_bench_seg_agg['RECUPERO'] / df_bench_seg_agg['CAPITAL']).astype(float)
    
    df_bench_seg_agg = df_bench_seg_agg[['PERIODO', 'AGENCIA', 'CARTERA', 'PRIORIDAD', 'CONTRATOS', 'CONTRATOS(%)', 'CAPITAL', 'CAPITAL(%)', 'RECUPERO', 'RECUPERO(%)', 'EFECTIVIDAD(%)']]
    
    df_bench_seg_agg_cartera = df_bench_seg_agg.groupby(['PERIODO', 'AGENCIA', 'CARTERA']).agg({
        'CONTRATOS': 'sum', 
        'CONTRATOS(%)': 'sum', 
        'CAPITAL': 'sum', 
        'CAPITAL(%)': 'sum', 
        'RECUPERO': 'sum', 
        'RECUPERO(%)': 'sum'
    }).reset_index()
    
    df_bench_seg_agg_cartera['EFECTIVIDAD(%)'] = (df_bench_seg_agg_cartera['RECUPERO'] / df_bench_seg_agg_cartera['CAPITAL']).astype(float)
    df_bench_seg_agg_cartera['PRIORIDAD'] = 'TOTAL'
    
    df_bench_seg_agg_cartera = df_bench_seg_agg_cartera[['PERIODO', 'AGENCIA', 'CARTERA', 'PRIORIDAD', 'CONTRATOS', 'CONTRATOS(%)', 'CAPITAL', 'CAPITAL(%)', 'RECUPERO', 'RECUPERO(%)', 'EFECTIVIDAD(%)']]
    
    df_bench_final_2 = pd.concat([df_bench_seg_agg, df_bench_seg_agg_cartera], ignore_index=True)
    df_bench_final_2.sort_values(by=['PERIODO', 'AGENCIA', 'CARTERA', 'PRIORIDAD'], inplace=True)
    df_bench_final_2.reset_index(drop=True, inplace=True)
    
    return df_bench_final_1, df_bench_final_2

In [ ]:
# Última Fecha Efectividades #

root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

periodos = obtener_periodos_año(datetime.now().year)
periodo_actual = periodos[-1]
periodo_anterior = periodos[-2]

result = messagebox.askquestion('Confirmación', f'¿Cargar periodo actual? \n\n SI: {periodo_actual} \n\n NO: {periodo_anterior}', icon='question')

if result == 'yes':
    periodo = periodo_actual
else:
    periodo = periodo_anterior

result = messagebox.askquestion('Confirmación', f'¿Obtener fecha de efectividad {periodo}?', icon='warning')

if result == 'yes':
    #conexion = get_connection(server_name='118.185.8.13', database_name='master')
    conexion = get_connection(server_name='118.180.55.67', database_name='CentroCobranzas', username='usrcobranza', password='P@ssw0rd')
    
    query = (
        f"""
            SELECT MAX(FECHA)
            FROM CENTROCOBRANZAS.DBO.PD_EFECTIVIDAD_TARDIA_HISTORICO
            WHERE PERIODO = {periodo}
        """
    )
    
    data = pd.read_sql(query, conexion)
    dia = data.iloc[0, 0][6:8]
    mes = data.iloc[0, 0][4:6]
    año = data.iloc[0, 0][:4]
    print(f'Máximo día de efectividades: {dia}/{mes}/{año}')
    conexion.close()

root.destroy()

In [ ]:
# Efectividades #

root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Conectar a SQL Server?', icon='question')

if result == 'yes':
    result = messagebox.askquestion('Confirmación', '¿Autenticación de Windows?', icon='warning')
    
    if result == 'yes':
        conexion: pyodbc.Connection = get_connection(server_name='118.185.8.13', database_name='master')
    else:
        conexion: pyodbc.Connection = get_connection(server_name='118.180.55.67', database_name='CentroCobranzas', username='usrcobranza', password='P@ssw0rd')
        #conexion: pyodbc.Connection = get_connection(server_name='118.180.55.67', database_name='CentroCobranzas', username='usrcobranzaext', password='3str4t3g14$GI')
    
    periodos = obtener_periodos_año(datetime.now().year)
    periodo_actual = periodos[-1]
    periodo_anterior = periodos[-2]
    
    result = messagebox.askquestion(
        'Confirmación', 
        f'¿Cargar periodo actual? \n\n SI: {periodo_actual} \n\n NO: {periodo_anterior}', 
        icon='question'
    )
    
    if result == 'yes':
        periodo = periodo_actual
    else:
        periodo = periodo_anterior
    
    query = (
    f"""
        SELECT
            PERIODO, 
            AGENCIA, 
            CODCENT AS CC, 
            CONTRATO, 
            CLAVE AS CARTERA, 
            CLUSTER, 
            FOCO AS PRIORIDAD, 
            CAPITALSOLES AS CAPITAL, 
            PAGOEFECTTOTALSOLESAGENCIA AS RECUPERO, 
            SEGMENTO_RIESGO AS SEGMENTO, 
            AMBITO_RCD_FINAL AS PRODUCTO, 
            INTENSIDAD AS INTENSIDAD_TOTAL, 
            DIRECTO_CALL AS INTENSIDAD_DIRECTA, 
            COBERTURA, 
            PDP, 
            PDP_CUMPLIDA
        FROM CENTROCOBRANZAS.DBO.PD_EFECTIVIDAD_TARDIA_HISTORICO
        WHERE 1 = 1 
        AND PERIODO = {periodo} 
        AND FECHA = (SELECT MAX(FECHA)
                    FROM CENTROCOBRANZAS.DBO.PD_EFECTIVIDAD_TARDIA_HISTORICO
                    WHERE PERIODO = {periodo})
    """
    )
    
    data = pd.read_sql(query, conexion)
    print(data.shape)
    data.to_csv(f'bases/efectividades/{periodo}/Efectividades_{periodo}.csv', index=False, sep=';', encoding='utf-8')
    conexion.close()

root.destroy()

In [ ]:
# Pagos Bench #

root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Conectar a SQL Server?', icon='question')

if result == 'yes':
    result = messagebox.askquestion('Confirmación', '¿Autenticación de Windows?', icon='warning')
    
    if result == 'yes':
        conexion: pyodbc.Connection = get_connection(server_name='118.185.8.13', database_name='master')
    else:
        conexion: pyodbc.Connection = get_connection(server_name='118.180.55.67', database_name='CentroCobranzas', username='usrcobranza', password='P@ssw0rd')
        #conexion: pyodbc.Connection = get_connection(server_name='118.180.55.67', database_name='CentroCobranzas', username='usrcobranzaext', password='3str4t3g14$GI')
    
    periodos = obtener_periodos_año(datetime.now().year)
    periodo_actual = periodos[-1]
    periodo_anterior = periodos[-2]
    
    result = messagebox.askquestion(
        'Confirmación', 
        f'¿Cargar periodo actual? \n\n SI: {periodo_actual} \n\n NO: {periodo_anterior}', 
        icon='question'
    )
    
    if result == 'yes':
        periodo = periodo_actual
    else:
        periodo = periodo_anterior
    
    query = (
    f"""
        SELECT
            PERIODO, 
            CLAVE AS CARTERA,
            AGENCIA, 
            CODCENT AS CC, 
            CONTRATO, 
            FOCO AS PRIORIDAD, 
            CAPITALSOLES AS CAPITAL, 
            PAGOEFECTTOTALSOLESAGENCIA AS RECUPERO_TOTAL,
            PAGOEFECTTOTALSOLESAGENCIACONT AS RECUPERO_CONTACTO
        FROM CENTROCOBRANZAS.DBO.PD_EFECTIVIDAD_TARDIA_HISTORICO
        WHERE 1 = 1 
        AND agencia IN ('ASESCOM RJ', 'CLASA MORA', 'MORNESE MORA')
        AND CLAVE IN ('BC', 'EXJ', 'KSTBC')
        AND FOCO IN ('C1', 'C2', 'C3')
        AND PERIODO = {periodo} 
        AND FECHA = (SELECT MAX(FECHA)
                    FROM CENTROCOBRANZAS.DBO.PD_EFECTIVIDAD_TARDIA_HISTORICO
                    WHERE PERIODO = {periodo})
    """
    )
    
    data = pd.read_sql(query, conexion)
    data.sort_values(by=['PERIODO', 'CARTERA', 'AGENCIA', 'PRIORIDAD'], inplace=True)
    print(data.shape)
    data.to_excel(f'bases/efectividades/{periodo}/Base_Pagos_{periodo}.xlsx', index=False)
    conexion.close()

root.destroy()

In [ ]:
# Generar Indicadores #

root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

# Primera pregunta
result = messagebox.askquestion(
    'Confirmación', 
    f'¿Cargar año actual? \n\n SI: {datetime.now().year} \n\n NO: {datetime.now().year - 1}', 
    icon='question'
)

if result == 'yes':
    año = datetime.now().year
else:
    año = datetime.now().year - 1

periodos = obtener_periodos_año(año)
ultimo_periodo = periodos[-1]

resultado_path = f'bases/efectividades/{ultimo_periodo}/Efectividades_Año_{ultimo_periodo[:4]}.xlsx'
print(resultado_path)


# Segunda pregunta
result = messagebox.askquestion(
    'Confirmación', 
    f'¿Cargar periodo actual? \n\n SI: {ultimo_periodo} \n\n NO: {periodos}', 
    icon='question'
)

if result == 'yes':
    periodos = [ultimo_periodo]

# Tercera pregunta
result = messagebox.askquestion('Confirmación', f'¿Comenzar proceso?', icon='question')

if result == 'yes':
    dataframes = leer_archivos_csv(periodos, 'bases/efectividades')
    df_metas = pd.read_excel('bases/metas/Metas.xlsx')
    
    resultado = {
        'Efectividades': pd.DataFrame(), 
        'Saldos': pd.DataFrame(), 
        'Indicadores': pd.DataFrame(), 
        'Bench Cartera': pd.DataFrame(), 
        'Bench Segmento': pd.DataFrame()
    }
    
    for periodo, df in dataframes.items():
        df_efect = dashboard_efectividades(df.copy())
        df_saldos = dashboard_saldos(df.copy())
        df_indicadores = dashboard_indicadores(df.copy())
        df_bench_1, df_bench_2 = dashboard_bench(df.copy(), df_metas)
        
        resultado['Efectividades'] = pd.concat([resultado['Efectividades'], df_efect], ignore_index=True)
        resultado['Saldos'] = pd.concat([resultado['Saldos'], df_saldos], ignore_index=True)
        resultado['Indicadores'] = pd.concat([resultado['Indicadores'], df_indicadores], ignore_index=True)
        resultado['Bench Cartera'] = pd.concat([resultado['Bench Cartera'], df_bench_1], ignore_index=True)
        resultado['Bench Segmento'] = pd.concat([resultado['Bench Segmento'], df_bench_2], ignore_index=True)
    
    with pd.ExcelWriter(resultado_path) as writer:
        resultado['Efectividades'].to_excel(writer, sheet_name='Efectividades', index=False)
        resultado['Saldos'].to_excel(writer, sheet_name='Saldos', index=False)
        resultado['Indicadores'].to_excel(writer, sheet_name='Indicadores', index=False)
        resultado['Bench Cartera'].to_excel(writer, sheet_name='Bench Cartera', index=False)
        resultado['Bench Segmento'].to_excel(writer, sheet_name='Bench Segmento', index=False)
    
    os.startfile(os.path.abspath(resultado_path))

root.destroy()